# Precalculated data for WDPA: biodiversity and contextual
This notebook merges the biodiversity and contextual data precalculated in ArcGIS Pro for the WDPA. Note that the biodiversity data needs to be formatted in an array of strings to be used by the front-end.
## Setup
### Library import
We import all the required Python libraries

In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import arcgis
from arcgis.gis import GIS
import json
import pandas as pd
from arcgis.features import FeatureLayerCollection
import requests as re
from copy import deepcopy
from itertools import repeat
import functools

### Utils

**getHTfromId**

In [2]:
def getHTfromId(item_id):
    item = gis.content.get(item_id)
    flayer = item.tables[0]
    sdf = flayer.query().sdf
    return sdf

**format_df**

In [3]:
def format_df(path, file_name, lookups_id):
    #save_steps= np.arange(1,217486,step=5000)
    ### Loop over each set of data and add all results to one dictionary that will be a text field in the Feature Service

    df = pd.read_csv(f'{path}/{file_name}')
    col_name = [col for col in df.columns if col in ['amphibians','birds','presence','reptiles']]
    df.rename(columns={'SliceNumbe':'SliceNumber',col_name[0]:'SUM'}, inplace=True)

    ### Get species area against global species range:
    lookup = getHTfromId(lookups_id)
    df = df.merge(lookup[['SliceNumber','range_area_km2']], how='left',on = 'SliceNumber')
    df['per_global'] = round(df['SUM']/df['range_area_km2']*100,2)
    df.loc[df['per_global']> 100,'per_global'] = 100 ### make max presence 100%
    
    ### Get species area against aoi area:
    df = df.merge(wdpa[['MOL_ID','AREA_KM2']])
    df['per_aoi'] = round(df['SUM']/df['AREA_KM2']*100,2)
    df.loc[df['per_aoi']> 100,'per_aoi'] = 100 ### make max presence 100%
    
    return df

## Connect to ArcGIS API

In [4]:
env_path = ".env"
with open(env_path) as f:
   env = {}
   for line in f:
       env_key, _val = line.split("=")
       env_value = _val.split("\n")[0]
       env[env_key] = env_value

In [5]:
aol_password = env['ARCGIS_GRETA_PASS']
aol_username = env['ARCGIS_GRETA_USER']

In [6]:
gis = GIS("https://eowilson.maps.arcgis.com", aol_username, aol_password, profile = "eowilson")

Keyring backend being used (keyring.backends.OS_X.Keyring (priority: 5)) either failed to install or is not recommended by the keyring project (i.e. it is not secure). This means you can not use stored passwords through GIS's persistent profiles. Note that extra system-wide steps must be taken on a Linux machine to use the python keyring module securely. Read more about this at the keyring API doc (http://bit.ly/2EWDP7B) and the ArcGIS API for Python doc (http://bit.ly/2CK2wG8).


## Read data

In [2]:
path_in = '/Users/sofia/Documents/HE_Data/WDPA_Precalculated/Inputs'
path_out = '/Users/sofia/Documents/HE_Data/WDPA_Precalculated/Outputs'

### WDPA

In [8]:
# wdpa table
wdpa= gpd.read_file(f'zip://{path_in}/WDPA_FILTERED_20210615_FILTERED_nomarine_with_oecm_wdpa.zip')
wdpa.rename(columns={'AREA_KM':'AREA_KM2'},inplace=True)

In [9]:
# Create new dataframe with wdpa information but excluding geometries. 
dff = wdpa[['NAME','MOL_ID','AREA_KM2', 'WDPAID','DESIG','DESIG_E','DESIG_T','IUCN_CA','ISO3']].copy() 
dff.head()

,NAME,MOL_ID,AREA_KM2,WDPAID,DESIG,DESIG_E,DESIG_T,IUCN_CA,ISO3
0,Boulder Beach,1,1.136031,310492.0,Stewardship Area,Stewardship Area,National,III,NZL
1,Ferndale,2,0.748492,307797.0,Scenic Reserve,Scenic Reserve,National,III,NZL
2,Broughton Bay,3,0.031907,307745.0,Scenic Reserve,Scenic Reserve,National,III,NZL
3,Kaipupu Point,4,0.270855,307867.0,Scenic Reserve,Scenic Reserve,National,III,NZL
4,Catlins Conservation Park,5,8.412168,303963.0,Stewardship Area,Stewardship Area,National,III,NZL


### WDPA's biodiversity data

In [10]:
### Ids of lookup tables for each taxa in ArcGIS online (run by Tamara in 2021)

lookups = {'amphibians':'c221a727e12048b2a6ec8e762bc5f478',
         'birds':'bcb31fd9091446a0af3cfdaed334a8da',
         'mammals':'212a3dd4665845deb5d2adf5b597aae0',
         'reptiles':'5b606a03b3fc431e8d4b9191c88bc2b9'}

# Get data for all taxa (calculated in arcgis pro with sample tool and saved in local)
amphibians = format_df(path_in, 'wdpa_amphibians_final_20211003.csv', lookups['amphibians'])
birds = format_df(path_in, 'wdpa_birds_final_20211003.csv', lookups['birds'])
mammals = format_df(path_in, 'wdpa_mammals_final_20211003.csv', lookups['mammals'])
reptiles = format_df(path_in, 'wdpa_reptiles_final_20211003.csv', lookups['reptiles'])

In [11]:
amphibians.head(1)

,OBJECTID,MOL_ID,X,Y,SUM,SliceNumber,Dimensions,range_area_km2,per_global,AREA_KM2,per_aoi
0,1,102,173.412225,-41.161038,1.0,3318,SliceNumber,296432,0.0,0.008886,100.0


In [31]:
birds.head(1)

,OID_,MOL_ID,X,Y,SUM,SliceNumber,Dimensions,range_area_km2,per_global,AREA_KM2,per_aoi
0,1,121,169.34608,-44.652219,1.0,251.0,SliceNumber,12099814,0.0,1.93288,51.74


In [32]:
mammals.head(1)

,OID_,MOL_ID,X,Y,SUM,SliceNumber,Dimensions,range_area_km2,per_global,AREA_KM2,per_aoi
0,1,131,172.506219,-41.971634,2.0,813.0,SliceNumber,86298,0.0,0.718592,100.0


In [33]:
reptiles.head(1)

,OBJECTID,MOL_ID,X,Y,SUM,SliceNumber,Dimensions,range_area_km2,per_global,AREA_KM2,per_aoi
0,1,121,169.34608,-44.652219,1.0,6163,SliceNumber,137822,0.0,1.93288,51.74


## Processing
### Format biodiversity data

In [11]:
%%time
amphibians = amphibians.groupby('MOL_ID')[['SliceNumber', 'per_global', 'per_aoi']].apply(lambda x: x.to_json(orient='records')).to_frame('amphibians').reset_index()
birds = birds.groupby('MOL_ID')[['SliceNumber', 'per_global', 'per_aoi']].apply(lambda x: x.to_json(orient='records')).to_frame('birds').reset_index()
mammals = mammals.groupby('MOL_ID')[['SliceNumber', 'per_global', 'per_aoi']].apply(lambda x: x.to_json(orient='records')).to_frame('mammals').reset_index()
reptiles = reptiles.groupby('MOL_ID')[['SliceNumber', 'per_global', 'per_aoi']].apply(lambda x: x.to_json(orient='records')).to_frame('reptiles').reset_index()


CPU times: user 38.1 s, sys: 9.42 s, total: 47.5 s
Wall time: 52 s


In [36]:
amphibians.head(1)

,MOL_ID,amphibians
0,2,"[{""SliceNumber"":3318,""per_global"":0.0,""per_aoi..."


In [12]:
dff = pd.merge(dff, amphibians, how='left', on = 'MOL_ID')
dff = pd.merge(dff, birds, how='left', on = 'MOL_ID')
dff = pd.merge(dff, mammals, how='left', on = 'MOL_ID')
dff = pd.merge(dff, reptiles, how='left', on = 'MOL_ID')
dff.head()

,NAME,MOL_ID,AREA_KM2,WDPAID,DESIG,DESIG_E,DESIG_T,IUCN_CA,ISO3,amphibians,birds,mammals,reptiles
0,Boulder Beach,1,1.136031,310492.0,Stewardship Area,Stewardship Area,National,III,NZL,NaN,"[{""SliceNumber"":482.0,""per_global"":0.0,""per_ao...","[{""SliceNumber"":303.0,""per_global"":0.0,""per_ao...",NaN
1,Ferndale,2,0.748492,307797.0,Scenic Reserve,Scenic Reserve,National,III,NZL,"[{""SliceNumber"":3318,""per_global"":0.0,""per_aoi...","[{""SliceNumber"":8.0,""per_global"":0.0,""per_aoi""...","[{""SliceNumber"":303.0,""per_global"":0.0,""per_ao...","[{""SliceNumber"":6163,""per_global"":0.0,""per_aoi..."
2,Broughton Bay,3,0.031907,307745.0,Scenic Reserve,Scenic Reserve,National,III,NZL,"[{""SliceNumber"":3318,""per_global"":0.0,""per_aoi...","[{""SliceNumber"":1847.0,""per_global"":0.0,""per_a...","[{""SliceNumber"":303.0,""per_global"":0.0,""per_ao...","[{""SliceNumber"":6499,""per_global"":0.0,""per_aoi..."
3,Kaipupu Point,4,0.270855,307867.0,Scenic Reserve,Scenic Reserve,National,III,NZL,"[{""SliceNumber"":3318,""per_global"":0.0,""per_aoi...","[{""SliceNumber"":8.0,""per_global"":0.0,""per_aoi""...","[{""SliceNumber"":303.0,""per_global"":0.0,""per_ao...","[{""SliceNumber"":6163,""per_global"":0.0,""per_aoi..."
4,Catlins Conservation Park,5,8.412168,303963.0,Stewardship Area,Stewardship Area,National,III,NZL,"[{""SliceNumber"":3318,""per_global"":0.0,""per_aoi...","[{""SliceNumber"":8.0,""per_global"":0.01,""per_aoi...",NaN,"[{""SliceNumber"":6163,""per_global"":0.01,""per_ao..."


In [13]:
dff = dff.fillna('[]')
dff.head()

,NAME,MOL_ID,AREA_KM2,WDPAID,DESIG,DESIG_E,DESIG_T,IUCN_CA,ISO3,amphibians,birds,mammals,reptiles
0,Boulder Beach,1,1.136031,310492.0,Stewardship Area,Stewardship Area,National,III,NZL,[],"[{""SliceNumber"":482.0,""per_global"":0.0,""per_ao...","[{""SliceNumber"":303.0,""per_global"":0.0,""per_ao...",[]
1,Ferndale,2,0.748492,307797.0,Scenic Reserve,Scenic Reserve,National,III,NZL,"[{""SliceNumber"":3318,""per_global"":0.0,""per_aoi...","[{""SliceNumber"":8.0,""per_global"":0.0,""per_aoi""...","[{""SliceNumber"":303.0,""per_global"":0.0,""per_ao...","[{""SliceNumber"":6163,""per_global"":0.0,""per_aoi..."
2,Broughton Bay,3,0.031907,307745.0,Scenic Reserve,Scenic Reserve,National,III,NZL,"[{""SliceNumber"":3318,""per_global"":0.0,""per_aoi...","[{""SliceNumber"":1847.0,""per_global"":0.0,""per_a...","[{""SliceNumber"":303.0,""per_global"":0.0,""per_ao...","[{""SliceNumber"":6499,""per_global"":0.0,""per_aoi..."
3,Kaipupu Point,4,0.270855,307867.0,Scenic Reserve,Scenic Reserve,National,III,NZL,"[{""SliceNumber"":3318,""per_global"":0.0,""per_aoi...","[{""SliceNumber"":8.0,""per_global"":0.0,""per_aoi""...","[{""SliceNumber"":303.0,""per_global"":0.0,""per_ao...","[{""SliceNumber"":6163,""per_global"":0.0,""per_aoi..."
4,Catlins Conservation Park,5,8.412168,303963.0,Stewardship Area,Stewardship Area,National,III,NZL,"[{""SliceNumber"":3318,""per_global"":0.0,""per_aoi...","[{""SliceNumber"":8.0,""per_global"":0.01,""per_aoi...",[],"[{""SliceNumber"":6163,""per_global"":0.01,""per_ao..."


In [14]:
dff.loc[dff['MOL_ID']==121,'birds'].values[0]

'[{"SliceNumber":251.0,"per_global":0.0,"per_aoi":51.74},{"SliceNumber":552.0,"per_global":0.0,"per_aoi":51.74},{"SliceNumber":613.0,"per_global":0.0,"per_aoi":51.74},{"SliceNumber":1301.0,"per_global":0.0,"per_aoi":51.74},{"SliceNumber":1310.0,"per_global":0.0,"per_aoi":51.74},{"SliceNumber":1321.0,"per_global":0.0,"per_aoi":51.74},{"SliceNumber":1510.0,"per_global":0.0,"per_aoi":51.74},{"SliceNumber":1511.0,"per_global":0.0,"per_aoi":51.74},{"SliceNumber":1517.0,"per_global":0.0,"per_aoi":51.74},{"SliceNumber":1572.0,"per_global":0.0,"per_aoi":51.74},{"SliceNumber":1933.0,"per_global":0.0,"per_aoi":51.74},{"SliceNumber":2017.0,"per_global":0.0,"per_aoi":51.74},{"SliceNumber":3025.0,"per_global":0.0,"per_aoi":51.74},{"SliceNumber":3027.0,"per_global":0.0,"per_aoi":51.74},{"SliceNumber":3081.0,"per_global":0.0,"per_aoi":51.74},{"SliceNumber":3277.0,"per_global":0.0,"per_aoi":51.74},{"SliceNumber":3422.0,"per_global":0.0,"per_aoi":51.74},{"SliceNumber":3552.0,"per_global":0.0,"per_aoi":

**Save table**

In [15]:
dff.to_csv((f'{path_out}/wdpa_precalculated_biodiversity_only.csv'))

### Add contextual data

In [16]:
pop = pd.read_csv(f'{path_in}/Pop.csv')
elu = pd.read_csv(f'{path_in}/ELU.csv')
le = pd.read_csv(f'{path_in}/LE.csv')

In [17]:
cont_lookups={'elu':'83802a7fa3d34c1fa40844fc14683966',
              'landencroach':'b0ad58e2b2404eec811b711b604b9d55'}

In [18]:
## Add contextual data: ELU
dff2 = dff.merge(elu[['MOL_ID','MAJORITY']], how='left', on = 'MOL_ID').rename(columns={'MAJORITY':'majority_land_cover_climate_regime'})
dff2.head()

,NAME,MOL_ID,AREA_KM2,WDPAID,DESIG,DESIG_E,DESIG_T,IUCN_CA,ISO3,amphibians,birds,mammals,reptiles,majority_land_cover_climate_regime
0,Boulder Beach,1,1.136031,310492.0,Stewardship Area,Stewardship Area,National,III,NZL,[],"[{""SliceNumber"":482.0,""per_global"":0.0,""per_ao...","[{""SliceNumber"":303.0,""per_global"":0.0,""per_ao...",[],107.0
1,Ferndale,2,0.748492,307797.0,Scenic Reserve,Scenic Reserve,National,III,NZL,"[{""SliceNumber"":3318,""per_global"":0.0,""per_aoi...","[{""SliceNumber"":8.0,""per_global"":0.0,""per_aoi""...","[{""SliceNumber"":303.0,""per_global"":0.0,""per_ao...","[{""SliceNumber"":6163,""per_global"":0.0,""per_aoi...",176.0
2,Broughton Bay,3,0.031907,307745.0,Scenic Reserve,Scenic Reserve,National,III,NZL,"[{""SliceNumber"":3318,""per_global"":0.0,""per_aoi...","[{""SliceNumber"":1847.0,""per_global"":0.0,""per_a...","[{""SliceNumber"":303.0,""per_global"":0.0,""per_ao...","[{""SliceNumber"":6499,""per_global"":0.0,""per_aoi...",NaN
3,Kaipupu Point,4,0.270855,307867.0,Scenic Reserve,Scenic Reserve,National,III,NZL,"[{""SliceNumber"":3318,""per_global"":0.0,""per_aoi...","[{""SliceNumber"":8.0,""per_global"":0.0,""per_aoi""...","[{""SliceNumber"":303.0,""per_global"":0.0,""per_ao...","[{""SliceNumber"":6163,""per_global"":0.0,""per_aoi...",NaN
4,Catlins Conservation Park,5,8.412168,303963.0,Stewardship Area,Stewardship Area,National,III,NZL,"[{""SliceNumber"":3318,""per_global"":0.0,""per_aoi...","[{""SliceNumber"":8.0,""per_global"":0.01,""per_aoi...",[],"[{""SliceNumber"":6163,""per_global"":0.01,""per_ao...",97.0


In [19]:
# Retrieve elu lookup table to see to see the correspondences for that elu code
elu_lookup = getHTfromId(cont_lookups['elu'])
elu_lookup.head()

,elu_code,elu,lc_type,lf_type,cr_type,ObjectId
0,301,Sub Tropical Moist Forest on Plains,Forest,Plains,Sub Tropical Moist,1
1,201,Warm Temperate Dry Sparsley or Non vegetated o...,Sparsley or Non vegetated,Plains,Warm Temperate Dry,2
2,151,Cool Temperate Dry Sparsley or Non vegetated o...,Sparsley or Non vegetated,Plains,Cool Temperate Dry,3
3,302,Sub Tropical Moist Cropland on Tablelands,Cropland,Tablelands,Sub Tropical Moist,4
4,152,Cool Temperate Dry Sparsley or Non vegetated o...,Sparsley or Non vegetated,Tablelands,Cool Temperate Dry,5


In [20]:
# Merge in dataset the required info from lookup table
dff2 = dff2.merge(elu_lookup[['elu_code','lc_type','cr_type']], how='left', left_on = 'majority_land_cover_climate_regime', right_on = 'elu_code')\
    .drop(columns=['elu_code'])\
    .rename(columns={'lc_type':'land_cover_majority','cr_type':'climate_regime_majority'})
dff2.head()

,NAME,MOL_ID,AREA_KM2,WDPAID,DESIG,DESIG_E,DESIG_T,IUCN_CA,ISO3,amphibians,birds,mammals,reptiles,majority_land_cover_climate_regime,land_cover_majority,climate_regime_majority
0,Boulder Beach,1,1.136031,310492.0,Stewardship Area,Stewardship Area,National,III,NZL,[],"[{""SliceNumber"":482.0,""per_global"":0.0,""per_ao...","[{""SliceNumber"":303.0,""per_global"":0.0,""per_ao...",[],107.0,Grassland,Cool Temperate Moist
1,Ferndale,2,0.748492,307797.0,Scenic Reserve,Scenic Reserve,National,III,NZL,"[{""SliceNumber"":3318,""per_global"":0.0,""per_aoi...","[{""SliceNumber"":8.0,""per_global"":0.0,""per_aoi""...","[{""SliceNumber"":303.0,""per_global"":0.0,""per_ao...","[{""SliceNumber"":6163,""per_global"":0.0,""per_aoi...",176.0,Forest,Warm Temperate Moist
2,Broughton Bay,3,0.031907,307745.0,Scenic Reserve,Scenic Reserve,National,III,NZL,"[{""SliceNumber"":3318,""per_global"":0.0,""per_aoi...","[{""SliceNumber"":1847.0,""per_global"":0.0,""per_a...","[{""SliceNumber"":303.0,""per_global"":0.0,""per_ao...","[{""SliceNumber"":6499,""per_global"":0.0,""per_aoi...",NaN,NaN,NaN
3,Kaipupu Point,4,0.270855,307867.0,Scenic Reserve,Scenic Reserve,National,III,NZL,"[{""SliceNumber"":3318,""per_global"":0.0,""per_aoi...","[{""SliceNumber"":8.0,""per_global"":0.0,""per_aoi""...","[{""SliceNumber"":303.0,""per_global"":0.0,""per_ao...","[{""SliceNumber"":6163,""per_global"":0.0,""per_aoi...",NaN,NaN,NaN
4,Catlins Conservation Park,5,8.412168,303963.0,Stewardship Area,Stewardship Area,National,III,NZL,"[{""SliceNumber"":3318,""per_global"":0.0,""per_aoi...","[{""SliceNumber"":8.0,""per_global"":0.01,""per_aoi...",[],"[{""SliceNumber"":6163,""per_global"":0.01,""per_ao...",97.0,Forest,Cool Temperate Moist


In [21]:
# See all land encroachment data for each mol_id
long =le[['SliceNumber','MOL_ID','percentage_land_encroachment']].pivot(index='MOL_ID', columns='SliceNumber', values='percentage_land_encroachment').reset_index().rename(columns=
                                                                                                                              {1.0:'percent_irrigated',
                                                                                                                              2.0:'percent_rainfed',
                                                                                                                              3.0:'percent_rangeland',
                                                                                                                                 4.0:'percent_urban'})

In [22]:
long.head()

SliceNumber,MOL_ID,percent_irrigated,percent_rainfed,percent_rangeland,percent_urban
0,1,NaN,NaN,100.00,NaN
1,4,NaN,NaN,100.00,NaN
2,5,NaN,NaN,83.33,NaN
3,6,NaN,NaN,50.00,NaN
4,9,NaN,NaN,107.69,NaN


In [23]:
# How many encroachment values are higher than 100? This can be due to the way the area is calculated. Anyway, the website will show only the le that is 
# majoritary and the % will stop at 100%
long['sum']=long.iloc[:, 1:5].sum(axis=1)
long[long['sum']>100]

SliceNumber,MOL_ID,percent_irrigated,percent_rainfed,percent_rangeland,percent_urban,sum
4,9,NaN,NaN,107.69,NaN,107.69
6,15,NaN,200.00,NaN,NaN,200.00
7,19,NaN,39.13,65.22,NaN,104.35
8,20,NaN,NaN,150.00,NaN,150.00
12,28,NaN,NaN,100.59,NaN,100.59
...,...,...,...,...,...,...
95962,217291,NaN,NaN,NaN,126.92,126.92
95963,217292,NaN,109.30,NaN,NaN,109.30
95965,217294,NaN,109.09,NaN,NaN,109.09
95983,217341,49.46,52.28,NaN,NaN,101.74


In [24]:
## Add contextual data: Land Encroachment
dff2 = dff2.merge(long[['MOL_ID','percent_irrigated', 'percent_rainfed', 'percent_rangeland','percent_urban']],on ='MOL_ID',how='left')

In [25]:
dff2.head()

,NAME,MOL_ID,AREA_KM2,WDPAID,DESIG,DESIG_E,DESIG_T,IUCN_CA,ISO3,amphibians,birds,mammals,reptiles,majority_land_cover_climate_regime,land_cover_majority,climate_regime_majority,percent_irrigated,percent_rainfed,percent_rangeland,percent_urban
0,Boulder Beach,1,1.136031,310492.0,Stewardship Area,Stewardship Area,National,III,NZL,[],"[{""SliceNumber"":482.0,""per_global"":0.0,""per_ao...","[{""SliceNumber"":303.0,""per_global"":0.0,""per_ao...",[],107.0,Grassland,Cool Temperate Moist,NaN,NaN,100.00,NaN
1,Ferndale,2,0.748492,307797.0,Scenic Reserve,Scenic Reserve,National,III,NZL,"[{""SliceNumber"":3318,""per_global"":0.0,""per_aoi...","[{""SliceNumber"":8.0,""per_global"":0.0,""per_aoi""...","[{""SliceNumber"":303.0,""per_global"":0.0,""per_ao...","[{""SliceNumber"":6163,""per_global"":0.0,""per_aoi...",176.0,Forest,Warm Temperate Moist,NaN,NaN,NaN,NaN
2,Broughton Bay,3,0.031907,307745.0,Scenic Reserve,Scenic Reserve,National,III,NZL,"[{""SliceNumber"":3318,""per_global"":0.0,""per_aoi...","[{""SliceNumber"":1847.0,""per_global"":0.0,""per_a...","[{""SliceNumber"":303.0,""per_global"":0.0,""per_ao...","[{""SliceNumber"":6499,""per_global"":0.0,""per_aoi...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Kaipupu Point,4,0.270855,307867.0,Scenic Reserve,Scenic Reserve,National,III,NZL,"[{""SliceNumber"":3318,""per_global"":0.0,""per_aoi...","[{""SliceNumber"":8.0,""per_global"":0.0,""per_aoi""...","[{""SliceNumber"":303.0,""per_global"":0.0,""per_ao...","[{""SliceNumber"":6163,""per_global"":0.0,""per_aoi...",NaN,NaN,NaN,NaN,NaN,100.00,NaN
4,Catlins Conservation Park,5,8.412168,303963.0,Stewardship Area,Stewardship Area,National,III,NZL,"[{""SliceNumber"":3318,""per_global"":0.0,""per_aoi...","[{""SliceNumber"":8.0,""per_global"":0.01,""per_aoi...",[],"[{""SliceNumber"":6163,""per_global"":0.01,""per_ao...",97.0,Forest,Cool Temperate Moist,NaN,NaN,83.33,NaN


In [26]:
## Add contextual data: POP
dff2 = dff2.merge(pop[['MOL_ID','SUM']],on ='MOL_ID',how='left')
dff2.head(1)

,NAME,MOL_ID,AREA_KM2,WDPAID,DESIG,DESIG_E,DESIG_T,IUCN_CA,ISO3,amphibians,...,mammals,reptiles,majority_land_cover_climate_regime,land_cover_majority,climate_regime_majority,percent_irrigated,percent_rainfed,percent_rangeland,percent_urban,SUM
0,Boulder Beach,1,1.136031,310492.0,Stewardship Area,Stewardship Area,National,III,NZL,[],...,"[{""SliceNumber"":303.0,""per_global"":0.0,""per_ao...",[],107.0,Grassland,Cool Temperate Moist,NaN,NaN,100.0,NaN,2.110001


In [27]:
dff2 = dff2.rename(columns ={'SUM':'population_sum'})
dff2.head(1)

,NAME,MOL_ID,AREA_KM2,WDPAID,DESIG,DESIG_E,DESIG_T,IUCN_CA,ISO3,amphibians,...,mammals,reptiles,majority_land_cover_climate_regime,land_cover_majority,climate_regime_majority,percent_irrigated,percent_rainfed,percent_rangeland,percent_urban,population_sum
0,Boulder Beach,1,1.136031,310492.0,Stewardship Area,Stewardship Area,National,III,NZL,[],...,"[{""SliceNumber"":303.0,""per_global"":0.0,""per_ao...",[],107.0,Grassland,Cool Temperate Moist,NaN,NaN,100.0,NaN,2.110001


In [41]:
dff2.head()

,NAME,MOL_ID,AREA_KM2,WDPAID,DESIG,DESIG_E,DESIG_T,IUCN_CA,ISO3,amphibians,...,mammals,reptiles,majority_land_cover_climate_regime,land_cover_majority,climate_regime_majority,percent_irrigated,percent_rainfed,percent_rangeland,percent_urban,population_sum
0,Boulder Beach,1,1.136031,310492.0,Stewardship Area,Stewardship Area,National,III,NZL,,...,"[{""SliceNumber"":303.0,""per_global"":0.0,""per_ao...",,107.0,Grassland,Cool Temperate Moist,NaN,NaN,100.00,NaN,2.110001
1,Ferndale,2,0.748492,307797.0,Scenic Reserve,Scenic Reserve,National,III,NZL,"[{""SliceNumber"":3318,""per_global"":0.0,""per_aoi...",...,"[{""SliceNumber"":303.0,""per_global"":0.0,""per_ao...","[{""SliceNumber"":6163,""per_global"":0.0,""per_aoi...",176.0,Forest,Warm Temperate Moist,NaN,NaN,NaN,NaN,1.315837
2,Broughton Bay,3,0.031907,307745.0,Scenic Reserve,Scenic Reserve,National,III,NZL,"[{""SliceNumber"":3318,""per_global"":0.0,""per_aoi...",...,"[{""SliceNumber"":303.0,""per_global"":0.0,""per_ao...","[{""SliceNumber"":6499,""per_global"":0.0,""per_aoi...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Kaipupu Point,4,0.270855,307867.0,Scenic Reserve,Scenic Reserve,National,III,NZL,"[{""SliceNumber"":3318,""per_global"":0.0,""per_aoi...",...,"[{""SliceNumber"":303.0,""per_global"":0.0,""per_ao...","[{""SliceNumber"":6163,""per_global"":0.0,""per_aoi...",NaN,NaN,NaN,NaN,NaN,100.00,NaN,NaN
4,Catlins Conservation Park,5,8.412168,303963.0,Stewardship Area,Stewardship Area,National,III,NZL,"[{""SliceNumber"":3318,""per_global"":0.0,""per_aoi...",...,,"[{""SliceNumber"":6163,""per_global"":0.01,""per_ao...",97.0,Forest,Cool Temperate Moist,NaN,NaN,83.33,NaN,3.103363


In [28]:
dff2['land_cover_majority'] = dff2['land_cover_majority'].fillna('')
dff2['climate_regime_majority'] = dff2['climate_regime_majority'].fillna('')
dff2 = dff2.fillna(0)
dff2.head()

,NAME,MOL_ID,AREA_KM2,WDPAID,DESIG,DESIG_E,DESIG_T,IUCN_CA,ISO3,amphibians,...,mammals,reptiles,majority_land_cover_climate_regime,land_cover_majority,climate_regime_majority,percent_irrigated,percent_rainfed,percent_rangeland,percent_urban,population_sum
0,Boulder Beach,1,1.136031,310492.0,Stewardship Area,Stewardship Area,National,III,NZL,[],...,"[{""SliceNumber"":303.0,""per_global"":0.0,""per_ao...",[],107.0,Grassland,Cool Temperate Moist,0.0,0.0,100.00,0.0,2.110001
1,Ferndale,2,0.748492,307797.0,Scenic Reserve,Scenic Reserve,National,III,NZL,"[{""SliceNumber"":3318,""per_global"":0.0,""per_aoi...",...,"[{""SliceNumber"":303.0,""per_global"":0.0,""per_ao...","[{""SliceNumber"":6163,""per_global"":0.0,""per_aoi...",176.0,Forest,Warm Temperate Moist,0.0,0.0,0.00,0.0,1.315837
2,Broughton Bay,3,0.031907,307745.0,Scenic Reserve,Scenic Reserve,National,III,NZL,"[{""SliceNumber"":3318,""per_global"":0.0,""per_aoi...",...,"[{""SliceNumber"":303.0,""per_global"":0.0,""per_ao...","[{""SliceNumber"":6499,""per_global"":0.0,""per_aoi...",0.0,,,0.0,0.0,0.00,0.0,0.000000
3,Kaipupu Point,4,0.270855,307867.0,Scenic Reserve,Scenic Reserve,National,III,NZL,"[{""SliceNumber"":3318,""per_global"":0.0,""per_aoi...",...,"[{""SliceNumber"":303.0,""per_global"":0.0,""per_ao...","[{""SliceNumber"":6163,""per_global"":0.0,""per_aoi...",0.0,,,0.0,0.0,100.00,0.0,0.000000
4,Catlins Conservation Park,5,8.412168,303963.0,Stewardship Area,Stewardship Area,National,III,NZL,"[{""SliceNumber"":3318,""per_global"":0.0,""per_aoi...",...,[],"[{""SliceNumber"":6163,""per_global"":0.01,""per_ao...",97.0,Forest,Cool Temperate Moist,0.0,0.0,83.33,0.0,3.103363


**Save table**

In [33]:
dff2.to_csv(f"{path_out}/wdpa_precal_with_contextual.csv")

In [3]:
dff = pd.read_csv(f"{path_out}/wdpa_precal_with_contextual.csv")

In [4]:
dff.head()

,Unnamed: 0,NAME,MOL_ID,AREA_KM2,WDPAID,DESIG,DESIG_E,DESIG_T,IUCN_CA,ISO3,...,mammals,reptiles,majority_land_cover_climate_regime,land_cover_majority,climate_regime_majority,percent_irrigated,percent_rainfed,percent_rangeland,percent_urban,population_sum
0,0,Boulder Beach,1,1.136031,310492.0,Stewardship Area,Stewardship Area,National,III,NZL,...,"[{""SliceNumber"":303.0,""per_global"":0.0,""per_ao...",[],107.0,Grassland,Cool Temperate Moist,0.0,0.0,100.00,0.0,2.110001
1,1,Ferndale,2,0.748492,307797.0,Scenic Reserve,Scenic Reserve,National,III,NZL,...,"[{""SliceNumber"":303.0,""per_global"":0.0,""per_ao...","[{""SliceNumber"":6163,""per_global"":0.0,""per_aoi...",176.0,Forest,Warm Temperate Moist,0.0,0.0,0.00,0.0,1.315837
2,2,Broughton Bay,3,0.031907,307745.0,Scenic Reserve,Scenic Reserve,National,III,NZL,...,"[{""SliceNumber"":303.0,""per_global"":0.0,""per_ao...","[{""SliceNumber"":6499,""per_global"":0.0,""per_aoi...",0.0,NaN,NaN,0.0,0.0,0.00,0.0,0.000000
3,3,Kaipupu Point,4,0.270855,307867.0,Scenic Reserve,Scenic Reserve,National,III,NZL,...,"[{""SliceNumber"":303.0,""per_global"":0.0,""per_ao...","[{""SliceNumber"":6163,""per_global"":0.0,""per_aoi...",0.0,NaN,NaN,0.0,0.0,100.00,0.0,0.000000
4,4,Catlins Conservation Park,5,8.412168,303963.0,Stewardship Area,Stewardship Area,National,III,NZL,...,[],"[{""SliceNumber"":6163,""per_global"":0.01,""per_ao...",97.0,Forest,Cool Temperate Moist,0.0,0.0,83.33,0.0,3.103363
